# 進階的 SQL 五十道練習

> 資料控制語言與交易控制語言

[數聚點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

## SQL 的組成與分類（複習）

- 具體來說，SQL 是由保留字（Keyword）、符號、常數與函數所組合而成的一種語言。
- 依照使用目的進而區別 SQL 的分類：

|SQL 的分類|範例|
|:---------|:----|
|資料查詢語言（Data Query Language, DQL）|`SELECT ...`|
|資料定義語言（Data Definition Language, DDL）|`CREATE ...`|
|資料操作語言（Data Manipulation Language, DML）|`UPDATE ...`|
|資料控制語言（Data Control Language, DCL）|`GRANT ...`|
|交易控制語言（Transaction Control Language, TCL）|`COMMIT ...`|

## 資料控制語言（Data Control Language, DCL）最主要的保留字

- `GRANT`
- `REVOKE`

這意味著我們可以給予及移除使用者的權限。

## 交易控制語言（Transaction Control Language, DCL）最主要的保留字

- `COMMIT`
- `ROLLBACK`

這意味著我們可以執行連串的整合型資料操作或者取消連串的整合型資料操作。

## 資料控制語言

## 透過 root 帳號（MySQL 最大權限的管理者）規劃角色

- 管理者：administrator
- 活躍用戶：poweruser
- 一般用戶：viewer

## 以 `CREATE` 建立角色

```sql
CREATE ROLE role_name@connection_name;
```

```sql
CREATE ROLE 'administrator'@'localhost',
            'poweruser'@'localhost',
            'viewer'@'localhost';
```

## 以 `GRANT` 給予角色權限

```sql
GRANT privilege ON db_name.table_name TO role_name@connection_name;
```

```sql
GRANT ALL ON imdb.* TO 'administrator'@'localhost';
GRANT CREATE VIEW ON imdb.* TO 'poweruser'@'localhost';
GRANT SELECT ON imdb.* TO 'poweruser'@'localhost', 'viewer'@'localhost';
```

## 以 `CREATE` 建立使用者

```sql
CREATE USER user_name@connection_name IDENTIFIED BY password DEFAULT ROLE role_name;
```

```sql
CREATE USER 'administrator1'@'localhost' IDENTIFIED BY 'password' DEFAULT ROLE 'administrator'@'localhost';
CREATE USER 'administrator2'@'localhost' IDENTIFIED BY 'password' DEFAULT ROLE 'administrator'@'localhost';
CREATE USER 'poweruser1'@'localhost' IDENTIFIED BY 'password' DEFAULT ROLE 'poweruser'@'localhost';
CREATE USER 'poweruser2'@'localhost' IDENTIFIED BY 'password' DEFAULT ROLE 'poweruser'@'localhost';
CREATE USER 'viewer1'@'localhost' IDENTIFIED BY 'password' DEFAULT ROLE 'viewer'@'localhost';
CREATE USER 'viewer2'@'localhost' IDENTIFIED BY 'password' DEFAULT ROLE 'viewer'@'localhost';
```

## 以不同的使用者帳號登入觀察權限

![](https://raw.githubusercontent.com/datainpoint/classroom-adv-sqlfifty/main/images/connection_dropdown.png)

## 以不同的使用者帳號登入觀察權限（續）

- 點選 Driver properties
- 調整 `allowPublicKeyRetrieval = TRUE`

![](https://raw.githubusercontent.com/datainpoint/classroom-adv-sqlfifty/main/images/connection_profile.png)

## 以 `REVOKE` 移除角色的權限

```sql
REVOKE privilege ON db_name.table_name FROM role_name@connection_name;
```

```sql
REVOKE CREATE VIEW ON imdb.* FROM 'poweruser'@'localhost';
```

## 以 `poweruser` 角色的帳號登入觀察權限

## 交易控制語言

## 將資料庫的多個操作合併執行就稱為交易（Transaction）

- 先前 SQL 的使用場景多半為「逐句執行」。
- 當我們需要連續新增、更新資料時，就可以透過交易將連串的操作整合為一個動作。
- 透過交易可以避免因操作過程中斷導致資料的不完整。

## 交易的特性

- 原子性（Atomicity）：交易中的操作一定會是「皆未執行」或者「皆已執行」其中一個狀態。
- 一致性（Consistency）：確保資料的完整性。
- 隔離性（Isolation）：執行到一半的操作並不會對其他操作產生影響。
- 持久性（Durability）：交易完成後不會遺失操作結果。

## 以帳戶資料為例

```sql
CREATE DATABASE tcl;
USE tcl;
CREATE TABLE accounts (
    id INT AUTO_INCREMENT,
    balance DECIMAL,
    PRIMARY KEY (id)
);
CREATE TABLE trades (
    id INT AUTO_INCREMENT,
    from_account_id INT,
    to_account_id INT,
    amount DECIMAL,
    PRIMARY KEY (id),
    FOREIGN KEY (from_account_id) REFERENCES accounts (id),
    FOREIGN KEY (to_account_id) REFERENCES accounts (id)
);
INSERT INTO accounts (balance) VALUES
	(10000),
	(10000);
```

## 描述與執行交易

- 以 `START TRANSACTION` 描述交易內容。
- 以 `COMMIT` 執行交易。

```sql
START TRANSACTION;
transactions...
COMMIT;
```

```sql
START TRANSACTION;
INSERT INTO trades (from_account_id, to_account_id, amount) VALUES
    (1, 2, 5000);
UPDATE accounts
   SET balance = balance - 5000
 WHERE id = 1;
UPDATE accounts
   SET balance = balance + 5000
 WHERE id = 2;
COMMIT;
```

## 描述與取消交易

- 以 `START TRANSACTION` 描述交易內容。
- 以 `ROLLBACK` 取消交易。

```sql
START TRANSACTION;
transactions...
ROLLBACK;
```

```sql
START TRANSACTION;
INSERT INTO trades (from_account_id, to_account_id, amount) VALUES
    (1, 2, 5000);
UPDATE accounts
   SET balance = balance - 5000
 WHERE id = 1;
UPDATE accounts
   SET balance = balance + 5000
 WHERE id = 2;
```

## 帳戶餘額以及交易明細已經更新

```sql
SELECT *
  FROM accounts;
SELECT *
  FROM trades;
```

## 以 `ROLLBACK` 取消交易後再檢視資料表

```sql
ROLLBACK;
SELECT *
  FROM accounts;
SELECT *
  FROM trades;
```